### 사전 준비

In [1]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 3.8 MB/s eta 0:00:00
--2023-02-15 06:52:54--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.08s   

2023-02-15 06:52:54 (60.1 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2023-02-15 06:52:54--  https://raw.githubusercontent.com/e9t/nsmc/mast

In [2]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
import numpy as np
import random
import os

In [3]:
torch.cuda.empty_cache()

device = torch.device("cuda:0")
print(device)

cuda:0


In [4]:
def fix_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

my_seed = 42
fix_seed(my_seed)

In [5]:
def seed_worker(_worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

### 데이터 load 및 증강

In [6]:
filenames = ['ratings_train.txt', 'ratings_test.txt']
with open('rating.txt', 'w') as outfile:
    for filename in filenames:
        with open(filename) as file:
            outfile.write(file.read())

In [7]:
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file, idx1, idx2):
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.dataset=self.dataset.sample(frac=1, random_state=my_seed).reset_index(drop=True)
    self.dataset = self.dataset[idx1:idx2]
    
    self.label = self.dataset['label'].astype(int).tolist()
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = int(row[1])

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=128,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [8]:
train_dataset = NSMCDataset("rating.txt",0,8000)
val_dataset = NSMCDataset("rating.txt",8000,9000)
test_dataset = NSMCDataset("rating.txt",9000,10000)

In [9]:
# 데이터 예시

train_dataset[1]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(tensor([    2,   532, 29953,  5891,  8927,     5,  1657, 20669, 30081,  5694,
         30422, 30751,  8087, 29948, 30035, 30041,  7083, 29053, 29956,  1290,
         29951,   934, 29957, 21176, 29990,   195, 29961,   494,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

### 모델 load 및 fine-tuning

In [10]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [11]:
epochs = 5
batch_size =64

In [12]:
g = torch.Generator()
g.manual_seed(0)

optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, worker_init_fn=seed_worker, generator=g)
val_loader = DataLoader(val_dataset, batch_size=batch_size, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_dataset, batch_size=batch_size, worker_init_fn=seed_worker, generator=g)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
losses = []
accuracies = []

for i in range(epochs):
  print("train {} epochs start!".format(i+1))
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0
  val_batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 40 == 0:
      print("Batch Loss:", loss, "Accuracy:", correct.float() / total)
    losses.append(loss)

  print("validation start!")
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(val_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    val_batches += 1
    if val_batches % 10 == 0:
      print("Batch Loss:", loss, "Accuracy:", correct.float() / total)

  accuracies.append(correct.float() / total)
  print("Train Loss:", loss, "Accuracy:", correct.float() / total)

train 1 epochs start!


  0%|          | 0/125 [00:00<?, ?it/s]

Batch Loss: tensor(0.6887, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5414, device='cuda:0')
Batch Loss: tensor(0.6798, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5877, device='cuda:0')
Batch Loss: tensor(0.6664, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.6262, device='cuda:0')
validation start!


  0%|          | 0/16 [00:00<?, ?it/s]

Batch Loss: tensor(0.6514, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.6353, device='cuda:0')
Train Loss: tensor(0.6613, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.6371, device='cuda:0')
train 2 epochs start!


...

Batch Loss: tensor(0.3552, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8284, device='cuda:0')
Train Loss: tensor(0.5618, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8276, device='cuda:0')
train 5 epochs start!


  0%|          | 0/125 [00:00<?, ?it/s]

Batch Loss: tensor(0.3718, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8438, device='cuda:0')
Batch Loss: tensor(0.3729, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8455, device='cuda:0')
Batch Loss: tensor(0.5232, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8479, device='cuda:0')
validation start!


  0%|          | 0/16 [00:00<?, ?it/s]

Batch Loss: tensor(0.3370, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8466, device='cuda:0')
Train Loss: tensor(0.4688, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8456, device='cuda:0')


### Test data를 사용한 평가

In [14]:
model.eval()

predict = []
test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  predicts = []
  for i in predicted:
    predicts.append(i.item())
  predict.extend(predicts)

  0%|          | 0/16 [00:00<?, ?it/s]

In [15]:
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, confusion_matrix

print("accuracy:{}".format(accuracy_score(test_dataset.label, predict)))
print("f1-score:{}".format(f1_score(test_dataset.label, predict)))
print("precision:{}".format(precision_score(test_dataset.label, predict ,pos_label=1)))
print("recall:{}".format(recall_score(test_dataset.label, predict ,pos_label=1)))

accuracy:0.835
f1-score:0.8367952522255192
precision:0.8213592233009709
recall:0.8528225806451613
